In [1]:
import json, os, tqdm
import numpy as np
from dpestimator import HeadPoseEstimator
from IPython.display import display
import PIL.Image as Image
import cv2
import matplotlib.pyplot as plt
from visualize.vis_3d import render_camera

In [2]:
dataset_path = '/data2/chence/PanoHeadData/single_view_hq/dataset_v2_balanced_nohat.json'
pe_info_path = '/data2/chence/PanoHeadData/single_view_hq/align_images/pose_estimation_info.json'
with open(dataset_path, 'r') as f:
    dataset_json = json.load(f)
image_names = list(dataset_json.keys())
pe = HeadPoseEstimator(weights_file = 'assets/whenet_1x3x224x224_prepost.onnx')
pose_estimation_info = {}

In [3]:
for image_name in image_names[:10]:
    print(json.dumps(dataset_json[image_name], indent = 4))
    print(image_name)
    break

{
    "data_source": "FFHQ",
    "camera": [
        0.9199845194816589,
        -0.004887542687356472,
        0.391928493976593,
        -1.0582069158554077,
        0.02866552025079727,
        -0.9964868426322937,
        -0.07882832735776901,
        0.21283648908138275,
        0.3909051716327667,
        0.08360766619443893,
        -0.9166123270988464,
        2.474853277206421,
        0.0,
        0.0,
        0.0,
        1.0,
        4.2647,
        0.0,
        0.5,
        0.0,
        4.2647,
        0.5,
        0.0,
        0.0,
        1.0
    ],
    "hpose": [
        -23.012277603149414,
        1.7729947566986084,
        -5.218562602996826
    ],
    "align_box": [
        139.2390243902439,
        139.55121951219513,
        738.0292682926829,
        738.0292682926829
    ],
    "align_quad": [
        [
            139.4185975609756,
            139.7737471417683
        ],
        [
            139.4185975609756,
            877.7139481707317
        ],
     

In [4]:
saved_json_path = '/data2/chence/PanoHeadData/single_view_hq/pose_estimation_info.json'
with open(saved_json_path, 'r') as f:
    saved_json_meta = json.load(f)

for image_name, image_meta in saved_json_meta.items():
    print(image_name)
    print(json.dumps(image_meta, indent=4))
    break

00000/img00000000.jpg
{
    "yaw": -22.181884765625,
    "yaw_bar": 21.44854736328125,
    "pitch": -6.54083251953125,
    "pitch_bar": -3.848846435546875,
    "roll": 2.8795013427734375,
    "roll_bar": -3.1965866088867188
}


In [5]:
vis_num = 5
losses = []
vised = 0
thresh = 10

failed = {'front':[], 'back':[], 'left':[], 'right':[]}
success = {'front':[], 'back':[], 'left':[], 'right':[]}
for image_name, image_meta in tqdm.tqdm(dataset_json.items()):
    view = dataset_json[image_name]['view']
    # if view != 'back': continue
    campose = dataset_json[image_name]['camera']
    image_name = image_name.replace('.png', '.jpg')
    
    # image_path = os.path.join('/data2/chence/PanoHeadData/single_view_hq/align_images', image_name)
    # image_data = cv2.imread(image_path)
    # image_data_flipped = cv2.flip(image_data, 1)
    # image_data_rgb = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    # image_data_flipped_rgb = cv2.cvtColor(image_data_flipped, cv2.COLOR_BGR2RGB)
    # camera_vis = render_camera(np.array(campose))

    yaw, roll, pitch = saved_json_meta[image_name]['yaw'], saved_json_meta[image_name]['roll'], saved_json_meta[image_name]['pitch']
    yaw_bar, roll_bar, pitch_bar = saved_json_meta[image_name]['yaw_bar'], saved_json_meta[image_name]['roll_bar'], saved_json_meta[image_name]['pitch_bar']
    yaw_bar = -yaw_bar if yaw_bar * yaw > 0 else yaw_bar
    roll_bar = -roll_bar if roll_bar * roll > 0 else roll_bar
    pitch_bar = -pitch_bar if pitch_bar * pitch < 0 else pitch_bar

    yaw_loss = np.sqrt((yaw + yaw_bar)**2)
    roll_loss = np.sqrt((roll + roll_bar)**2)
    pitch_loss = np.sqrt((pitch - pitch_bar)**2)
    loss = np.sqrt(
        (1  *(pitch - pitch_bar))**2 + 
        (1  *(roll + roll_bar))**2 + 
        (1  *(yaw + yaw_bar))**2
    )
    if loss < thresh: 
        losses.append(loss)
        success[view].append((image_name, campose, loss))
    else: 

        # fig, ax = plt.subplots(1, 3, figsize = (12, 4))
        # ax[0].imshow(image_data_rgb)
        # ax[1].imshow(camera_vis)
        # ax[2].imshow(image_data_flipped_rgb)
        # fig.suptitle(f'{image_name}, loss: {loss:.2f}')
        # display(fig)
        
        failed[view].append((image_name, campose, loss))
    #     vis_num += 1
    # if vis_num == vised: break
for key, value in success.items():
    print(key, len(value))
# for key, value in failed.items():
#     print(key, len(value))

100%|██████████| 404524/404524 [00:04<00:00, 91988.07it/s]

front 160140
back 49636
left 32397
right 37464


In [8]:
vis_num = 10
filtered_cam_dict = {}
for _view in ['front', 'back', 'left', 'right']:
    for image_name, campose, loss in success[_view]:
        filtered_cam_dict[image_name] = dataset_json[image_name.replace('.jpg', '.png')]
print(len(filtered_cam_dict))
with open('/data2/chence/PanoHeadData/single_view_hq/dataset-cam-filtered.json', 'w') as f:
    json.dump(filtered_cam_dict, f, indent = 4)
    # image_path = os.path.join('/data2/chence/PanoHeadData/single_view_hq/align_images', image_name)
    # image_data = cv2.imread(image_path)
    # image_data_flipped = cv2.flip(image_data, 1)
    # image_data_rgb = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    # image_data_flipped_rgb = cv2.cvtColor(image_data_flipped, cv2.COLOR_BGR2RGB)
    # camera_vis = render_camera(np.array(campose))
    # fig, ax = plt.subplots(1, 3, figsize = (12, 4))
    # ax[0].imshow(image_data_rgb)
    # ax[1].imshow(camera_vis)
    # ax[2].imshow(image_data_flipped_rgb)
    # fig.suptitle(f'{image_name}, loss: {loss:.2f}')
    # display(fig)
    # vis_num -= 1
    # if vis_num == 0:break


279637
